In [1]:
# Set paths for the training data

base_dir = 'C:\\Users\\nins_\\Desktop\\Data3\\'
csv_file_name = 'driving_log.csv'

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Flatten, Dense, Convolution2D, MaxPooling2D, Activation, Cropping2D, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
%matplotlib inline

Using TensorFlow backend.


In [3]:
# Load data from csv and images into training and label numpy arrays
df = pd.read_csv(base_dir+csv_file_name, header=None)

In [4]:
# Split into training and validation sets
data_train, data_valid = train_test_split(df, test_size=0.2)

In [5]:
# Generator for supplying data to the model in batches

def data_generator(data, batch_size):
    
    data = np.array(data)
    total_samples = len(data)
    
    while(True):
        
        # Shuffle the data before each epoch
        shuffle(data)
        
        # Obtain samples equal to batch size and process the images
        for offset in range(0, total_samples, batch_size):
            batch = data[offset:batch_size+offset]
            
            batch_images = []
            batch_labels = []
            
            # Load images and labels equal to the batch size only
            for row in batch:
                img_C = cv2.cvtColor(cv2.imread(row[0]), cv2.COLOR_BGR2RGB)
                label_C = float(row[3])
                
                img_L = cv2.cvtColor(cv2.imread(row[1]), cv2.COLOR_BGR2RGB)
                op = 0.7 if label_C < 0 else 1.3
                label_L = op * label_C
                
                img_R = cv2.cvtColor(cv2.imread(row[2]), cv2.COLOR_BGR2RGB)
                op = 1.3 if label_C < 0 else 0.7
                label_R = op * label_C
                
                # Append to the batch data
                batch_images.append(img_C)
                batch_images.append(img_L)
                batch_images.append(img_R)
                
                batch_labels.append(label_C)
                batch_labels.append(label_L)
                batch_labels.append(label_R)
                
                # Flip the image
                flip_img_C = cv2.flip(img_C, 1)
                flip_label_C = label_C * (-1)
                
                flip_img_L = cv2.flip(img_L, 1)
                flip_label_L = label_L * (-1)
                
                flip_img_R = cv2.flip(img_R, 1)
                flip_label_R = label_R * (-1)
                
                # Append the augmented data to the batch data
                batch_images.append(flip_img_C)
                batch_images.append(flip_img_L)
                batch_images.append(flip_img_R)
                
                batch_labels.append(flip_label_C)
                batch_labels.append(flip_label_L)
                batch_labels.append(flip_label_R)
                
                
                
            
            X = np.array(batch_images)
            y = np.array(batch_labels)

            yield shuffle(X, y)

In [6]:
# Create generators for training and validation

train_generator = data_generator(data_train, batch_size=16)
valid_generator = data_generator(data_valid, batch_size=16)

In [7]:
# Model Definition

model = Sequential()
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))

model.add(Convolution2D(8, 5, 5, border_mode='valid', subsample=(1,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation(activation='relu'))

model.add(Convolution2D(6, 3, 3, border_mode='valid', subsample=(1,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation(activation='relu'))

model.add(Convolution2D(6, 3, 3, border_mode='same', subsample=(1,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation(activation='relu'))

model.add(Flatten())
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(150, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
filepath="weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True)
earlystop = EarlyStopping(monitor='val_loss', patience=3)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_1 (Cropping2D)        (None, 90, 320, 3)    0           cropping2d_input_1[0][0]         
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 86, 316, 8)    608         cropping2d_1[0][0]               
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 43, 158, 8)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 43, 158, 8)    0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [8]:
# Train the model

model.fit_generator(train_generator, callbacks=[checkpoint, earlystop], samples_per_epoch=len(data_train), validation_data=valid_generator, nb_val_samples=len(data_valid), nb_epoch=40)
model.save('model.h5')

Epoch 1/40
2592/2679 [============================>.] - ETA: 1s - loss: 3090.8661

H:\Programming_Tools\Anaconda\envs\carnd-term1\lib\site-packages\keras\engine\training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


2688/2679 [==============================] - 42s - loss: 2981.0326 - val_loss: 0.4925
Epoch 2/40
2688/2679 [==============================] - 26s - loss: 4.0678 - val_loss: 0.0481
Epoch 3/40
2688/2679 [==============================] - 25s - loss: 0.3301 - val_loss: 0.0422
Epoch 4/40
2688/2679 [==============================] - 26s - loss: 0.1176 - val_loss: 0.0199
Epoch 5/40
2688/2679 [==============================] - 26s - loss: 0.0747 - val_loss: 0.0307
Epoch 6/40
2730/2679 [==============================] - 27s - loss: 0.0474 - val_loss: 0.0165
Epoch 7/40
2688/2679 [==============================] - 26s - loss: 0.0449 - val_loss: 0.0213
Epoch 8/40
2688/2679 [==============================] - 27s - loss: 0.0278 - val_loss: 0.0298
Epoch 9/40
2688/2679 [==============================] - 25s - loss: 0.0291 - val_loss: 0.0255
Epoch 10/40
2688/2679 [==============================] - 25s - loss: 0.0305 - val_loss: 0.0371
